<a href="https://colab.research.google.com/github/ayush-0210/Crowdsourced-Beer-Recommendation-System/blob/main/main_code.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

Team Members: 


1.   Sahil Arora (sa52643)
2.   Kaushik Kumaran (kk34537)
3.   Ayush Malani (am95655)
4.   Harsh Mehta (hdm564)
5.   Charan Musunuru (cm59982)
6.   Allie Touchstone (awt529)


## Task A: Scrape Data and Collect ~6000 Beer Reviews

This was done using webscraper.io.

## Task B: Attribute Selection using Word Frequencies

Importing the necessary libraries

In [ ]:
# !pip install pandas==1.3.3
from sklearn.preprocessing import MinMaxScaler
from sklearn.manifold import MDS
import pandas as pd
import numpy as np
import sys
import matplotlib.pyplot as plt
import pandas as pd
import nltk
from nltk import word_tokenize
nltk.download('stopwords')
nltk.download('punkt')
from nltk.corpus import stopwords
from collections import Counter
!pip install vaderSentiment

[nltk_data] Downloading package stopwords to /root/nltk_data...
[nltk_data]   Unzipping corpora/stopwords.zip.
[nltk_data] Downloading package punkt to /root/nltk_data...
[nltk_data]   Unzipping tokenizers/punkt.zip.


In [ ]:
# Uploading the data frame 
from google.colab import files
uploaded = files.upload()

Saving beeradvocate.csv to beeradvocate.csv


In [ ]:
beer_df = pd.read_csv('beeradvocate.csv')

In [ ]:
def word_counts(x):
    # Word tokenize your comment
    x = str(x)
    tokens = word_tokenize(x)
    # Collecting the stopwords
    stop_words = set(stopwords.words('english'))
    
    # Removing the stopwords
    tokens_rm_stp = []
    for token in tokens:
        lt = token.lower() 
        if lt not in stop_words:
            tokens_rm_stp.append(lt)
    
    # Creating a word frequency dictionary
    counter = Counter(tokens_rm_stp)
    return counter

In [ ]:
beer_count = pd.DataFrame(list(beer_df['product_review'].map(word_counts)))
beer_count = beer_count.fillna(0)
beer_counts = pd.DataFrame(beer_count.sum().sort_values(ascending = False))

In [ ]:
beer_counts.head(60)

,0
.,47491.0
",",45201.0
:,5273.0
beer,4994.0
head,3804.0
taste,3115.0
chocolate,2844.0
dark,2738.0
's,2697.0
-,2683.0


Attributes Chosen:

•	Aggressive: Boldly assertive aroma and/or taste <br>
•	Balanced: Equal representation of malt sweetness and hop bitterness in the flavor — especially at the finish <br>
•	Fruity: Flavors reminiscent of various fruits <br>

In [ ]:
# Replaced 'I want beer' with three atrributes. It is better if it is in a sentence.
y = 'Aggressive Balanced Fruity' 

In [ ]:
beer_df

,product_name,product_rating,product_review
0,Omega Point,4.72,22oz. capped bottle into a Brooklyn Brewery sn...
1,Trappistes Rochefort 10,4.15,"This is a wonderful Quad, but the alcohol note..."
2,Double Nelson,4.39,750mL growler from the brewery. Pours a hazy p...
3,Great,4.75,"Deep amber with a surprisingly thick, white he..."
4,West Ashley,3.86,Pours a cloudy orange colored beer with a medi...
...,...,...,...
6222,Flora,4.60,Florence is probably my favorite shelf Saison ...
6223,Juice Machine,4.42,Canned on 8/5/20. Poured into a gravity pint g...
6224,Double Sunshine,4.13,29JUL20 saw a second visit to Lawson's in 2 da...
6225,Imperial German Chocolate Cupcake Stout,4.41,750ml bottle poured into a snifter\n\n\n\nA- p...


In [ ]:
beer_df.dropna(inplace=True)

## Task C: Cosine Similarity

In [ ]:
from sklearn.metrics.pairwise import cosine_similarity
from sklearn.feature_extraction.text import TfidfVectorizer

def cos_sim(x):
    corpus = [x,y]
    vectorizer = TfidfVectorizer()
    trsfm = vectorizer.fit_transform(corpus)
    return cosine_similarity(trsfm[0], trsfm)[0][1]

In [ ]:
beer_df['sim_scores_with_attr'] = beer_df['product_review'].map(cos_sim)

In [ ]:
beer_df[beer_df['sim_scores_with_attr'] != 0.000000]

,product_name,product_rating,product_review,sim_scores_with_attr
5,Double Dry Hopped Congress Street,4.65,Had on tap at brewery August 2019\n\n\n\nWorld...,0.046395
7,Crusher,4.57,"No date on the can, but this one's pretty fres...",0.014030
8,Double Dry Hopped Congress Street,4.18,"1 pint can\n\nServed in an oversized, stemless...",0.027074
10,Black & Wild,4.66,8oz pour on draft at the brewery tasting room....,0.028544
17,Emerald Grouper,4.05,eyes...\n\nmassive carbonation erupts from thi...,0.012386
...,...,...,...,...
6209,Pliny The Younger,4.65,Stoked to finally get to try this one. My best...,0.011359
6217,Ephraim,4.60,"Bright, refereshing, peppery, orange citrus, p...",0.127847
6218,Notorious Triple IPA,4.86,L- pours a hazy orange color with a half inch ...,0.036559
6219,Speedway Stout - Bourbon Barrel-Aged,4.49,"Bottled 10/30/13.\n\n\n\nNice dark pour, not t...",0.023868


## Task D: Sentiment Analysis

In [ ]:
from vaderSentiment.vaderSentiment import SentimentIntensityAnalyzer
analyser = SentimentIntensityAnalyzer()
def get_sentiment(review, **kwargs):
    sentiment_score = analyser.polarity_scores(review)
    positive_meter = round((sentiment_score['pos'] * 100), 2)
    negative_meter = round((sentiment_score['neg'] * 100), 2)
    return positive_meter if kwargs['k'] == 'positive' else negative_meter

In [ ]:
beer_df['positive_sent_score'] = beer_df['product_review'].apply(get_sentiment, k='positive')
beer_df['negative_sent_score'] = beer_df['product_review'].apply(get_sentiment, k='negative')

In [ ]:
beer_df

,product_name,product_rating,product_review,sim_scores_with_attr,positive_sent_score,negative_sent_score
0,Omega Point,4.72,22oz. capped bottle into a Brooklyn Brewery sn...,0.000000,15.2,2.1
1,Trappistes Rochefort 10,4.15,"This is a wonderful Quad, but the alcohol note...",0.000000,19.1,13.4
2,Double Nelson,4.39,750mL growler from the brewery. Pours a hazy p...,0.000000,7.8,0.0
3,Great,4.75,"Deep amber with a surprisingly thick, white he...",0.000000,36.6,0.0
4,West Ashley,3.86,Pours a cloudy orange colored beer with a medi...,0.000000,15.7,4.2
...,...,...,...,...,...,...
6222,Flora,4.60,Florence is probably my favorite shelf Saison ...,0.000000,20.0,8.0
6223,Juice Machine,4.42,Canned on 8/5/20. Poured into a gravity pint g...,0.000000,6.6,10.0
6224,Double Sunshine,4.13,29JUL20 saw a second visit to Lawson's in 2 da...,0.000000,8.3,3.7
6225,Imperial German Chocolate Cupcake Stout,4.41,750ml bottle poured into a snifter\n\n\n\nA- p...,0.000000,13.7,2.3


In [ ]:
beer_df[beer_df['sim_scores_with_attr']==0].shape[0]

5049

## Task E: Evaluation Score

In [ ]:
score_df = beer_df.groupby('product_name')[['positive_sent_score','sim_scores_with_attr']].mean()
score_df['positive_sent_score_scaled'] = (score_df['positive_sent_score']  - score_df['positive_sent_score'] .min())/(score_df['positive_sent_score'].max() - score_df['positive_sent_score'].min())
score_df['final_score'] = score_df['positive_sent_score_scaled'] + score_df['sim_scores_with_attr']
score_df.sort_values(by='final_score',ascending=False)[:3]

,positive_sent_score,sim_scores_with_attr,positive_sent_score_scaled,final_score
product_name,,,,
DFPF,24.420,0.015545,1.000000,1.015545
Flora Plum,22.712,0.012354,0.855646,0.867999
Rare Scooop,22.488,0.001818,0.836714,0.838532


In [ ]:
import spacy 
!python -m spacy download en_core_web_md
# nlp = spacy.load("en_core_web_md")
import en_core_web_md
nlp = en_core_web_md.load()
import warnings; warnings.simplefilter('ignore')

     |████████████████████████████████| 96.4 MB 22.2 MB/s 
  Created wheel for en-core-web-md: filename=en_core_web_md-2.2.5-py3-none-any.whl size=98051302 sha256=bacbb9af17781318bcbb017cf5dd7fc33b4110ebfda4b33ed841754750da0fd5
  Stored in directory: /tmp/pip-ephem-wheel-cache-0_ft4qgt/wheels/69/c5/b8/4f1c029d89238734311b3269762ab2ee325a42da2ce8edb997
Successfully built en-core-web-md
✔ Download and installation successful
You can now load the model via spacy.load('en_core_web_md')


In [ ]:
user_comment = "Aggressive Balanced Fruity" # Balanced, Complex, Crisp, Fruity, Robust
doc2 = nlp(user_comment)

def find_similarity(review):
    review_nlp = nlp(review)
    sim_score = review_nlp.similarity(doc2)
    return sim_score
beer_df['spacy_score'] = beer_df['product_review'].apply(find_similarity)

In [ ]:
score_df = beer_df.groupby('product_name')[['positive_sent_score','spacy_score']].mean()
score_df['positive_sent_score_scaled'] = (score_df['positive_sent_score']  - score_df['positive_sent_score'] .min())/(score_df['positive_sent_score'].max() - score_df['positive_sent_score'].min())
score_df['final_score'] = score_df['positive_sent_score_scaled'] + score_df['spacy_score']
score_df.sort_values(by='final_score',ascending=False)[:3]
score_df.reset_index(inplace=True)

,positive_sent_score,spacy_score,positive_sent_score_scaled,final_score
product_name,,,,
DFPF,24.420,0.584696,1.000000,1.584696
Flora Plum,22.712,0.609049,0.855646,1.464694
Keene Idea,22.404,0.579929,0.829615,1.409543


In [ ]:
score_df

,product_name,positive_sent_score,spacy_score,positive_sent_score_scaled,final_score
0,A Deal With The Devil,17.152,0.596127,0.385734,0.981861
1,A Deal With The Devil - Double Oak-Aged,21.232,0.591269,0.730561,1.321830
2,Aaron,16.152,0.603580,0.301217,0.904797
3,Abner,17.964,0.594666,0.454361,1.049027
4,Abrasive Ale,16.800,0.608247,0.355984,0.964231
...,...,...,...,...,...
245,Westly,16.724,0.600138,0.349561,0.949698
246,Wide Awake It's Morning,15.892,0.601688,0.279243,0.880930
247,Zenne Y Frontera,19.560,0.607828,0.589249,1.197077
248,Zombie Dust,15.648,0.599928,0.258621,0.858549


## Task F and G: Recommendations

In [ ]:
# Top 3 rated beers from the dataset
score_df_top = beer_df.groupby('product_name')['product_rating'].mean()
score_df_top.sort_values(ascending=False)[:3]

product_name
Chemtrailmix             4.77160
Vanilla Bean Assassin    4.74625
Blessed                  4.74280
Name: product_rating, dtype: float64

In [ ]:
top_rated = score_df_top.sort_values(ascending=False)[:3].index

In [ ]:
score_df[score_df['product_name'].isin( top_rated)]

,product_name,positive_sent_score,spacy_score,positive_sent_score_scaled,final_score
38,Blessed,21.4960,0.585656,0.752874,1.338529
54,Chemtrailmix,16.3440,0.578975,0.317444,0.896420
238,Vanilla Bean Assassin,21.0625,0.577382,0.716236,1.293617



Insight
Beer Attributes considered - Aggressive, Balanced, Fruity

Our recommendations using Spacy Similarity:
1. DFPF 
2. Flora Plum
3. Keene Idea

Our recommendations are products that have both high average similarity scores and sentiment scores. It seems that spacy gives much more middle-of-the-road scores while the sentiment scores are more polarized in each direction. 


After manually glancing through the user reviews for these, we could find that people enjoy the fruity flavor in 'DFPF' and 'Flora Plum,' which backs up our user's 'fruity' preference. The same as our last recommendation - 'Keene Idea,' which is highly regarded for its aggressive and balanced taste. This suggests that our recommendations contain a balanced mix of fruity flavors and do justice to the user preference of 'balanced' and 'fruity.'


Top Rated Beers from the dataset:
1. Blessed 
2. Chemtrailmix 
3. Vanilla Bean Assassin

From the user reviews of these top-rated beers, we saw that the top-rated beers are not that bad of a recommendation. They all have around average positive sentiment scores and similarity scores with the attributes.
The spacy scores alone are very similar. The recommendations from the spacy similarity have an average spacy score of 0.59, and the recommendations from the dataset have an average spacy score of 0.57. However, compared with the sentiment scores where the spacy recommendations have an average sentiment score of 0.89, and the dataset recommendations have an average of 0.59, the spacy recommendations should be more favorable to the customer. 

We could infer that these three beers are reasonably good when considering the specific tastes they offer; however, our recommendations should provide the customer with options that better aline with their preferences. 